In [4]:
import os
os.chdir("../")

In [140]:
import os
import numpy as np
import pickle
import matplotlib
import matplotlib.pyplot as plt
from numpy.random import random, shuffle
%matplotlib inline

In [66]:
def one_hot_encoding(labels, bound):
    result = np.zeros((labels.size, bound))
    result[np.arange(labels.size), labels] = 1
    return result

In [294]:
def relu(vector):
    '''
    ReLU function Max{0, vector} for hidden layer
    '''
    return np.maximum(vector, 0, vector)

def softmax(X):
    '''
    Softmax function for output layer
    '''
    return np.exp(X) / np.sum(np.exp(X), axis=1, keepdims=True)

In [346]:
def loss_and_acc(X, Y, w, b):
    N = X.shape[0]
    Y_hat, _, _ = forward_prop(X, w[0], w[1], w[2], b[0], b[1], b[2])
    #print(Y_hat)
    #loss = -np.sum(Ys_true.T * np.log(Y_hats)) / N
    acc = (np.argmax(Y_hat, axis=1) ==
           np.argmax(Y, axis=1)).mean() * 100
    #return loss, acc
    print(acc)

In [422]:
def forward_prop(X, w1, w2, w3, b1, b2, b3):
    N = X.shape[0]
    z2 = X.dot(w1) + np.repeat(b1, N, axis=0)
    a2 = relu(z2)
    z3 = a2.dot(w2) + np.repeat(b2, N, axis=0)
    a3 = relu(z3)
    z4 = a3.dot(w3) + np.repeat(b3, N, axis=0)
    Y_hat = softmax(z4)
    #print("a", z2,a2,a3)
    return Y_hat, [z2,z3,z4], [X,a2,a3]

In [423]:
def d_relu(X):
    # 0-1 
    return np.where(X < 0, 0, X)

In [450]:
def back_prop(Y_hat, y_ture, w, b, a, z):
    N = Y_hat.shape[0]
    delta3 = - (y_ture - Y_hat)
    delta2 = (w[2].dot(delta3.T)).T*d_relu(z[1])
    delta1 = (w[1].dot(delta2.T)).T*d_relu(z[0])
    
    g_w3 = a[2].T.dot(delta3)/ N
    g_w2 = a[1].T.dot(delta2)/ N
    g_w1 = a[0].T.dot(delta1)/ N
    #print(a[2])
    #print(delta1.shape)
    g_b1 = np.sum(delta1, axis=0, keepdims=True) / N
    g_b2 = np.sum(delta2, axis=0, keepdims=True) / N
    g_b3 = np.sum(delta3, axis=0, keepdims=True) / N
    
    return [g_w1,g_w2,g_w3], [g_b1,g_b2,g_b3]

In [471]:
# Definition of functions and parameters
'''
matrix format
input matrx: sample_num*dimension_size
weight matrix: input_dimension*output_dimension
'''
# for example
EPOCH = 100
alpha = 0.02

# Read all data from .pkl
(train_images, train_labels, test_images, test_labels) = pickle.load(open('./mnist_data/data.pkl', 'rb'),encoding='latin1')
train_images = np.array(train_images)
train_labels = one_hot_encoding(np.array(train_labels), 10)
test_images = np.array(test_images)
test_labels = one_hot_encoding(np.array(test_labels), 10)

### 1. Data preprocessing: normalize all pixels to [0,1) by dividing 256
train_images = train_images/256
test_images = test_images/256

### 2. Weight initialization: Xavier
n1 = train_images.shape[1]
n2 = 300
n3 = 100
n4 = 10
w1 = (np.random.random((n1,n2))-0.5)*np.sqrt(6/(n1+n2))
w2 = (np.random.random((n2,n3))-0.5)*np.sqrt(6/(n2+n3))
w3 = (np.random.random((n3,n4))-0.5)*np.sqrt(6/(n3+n4))
b1 = np.zeros((1,n2))
b2 = np.zeros((1,n3))
b3 = np.zeros((1, n4))

In [472]:
### 3. training of neural network
loss = np.zeros((EPOCH))
accuracy = np.zeros((EPOCH))
training_size = train_images.shape[0]
all_idx = list(range(training_size))
batch_size = 100
for epoch in range(100):
    print(epoch)
    if epoch>50:
        alpha = 0.01
    shuffle(all_idx)
    for batch_idx in np.array_split(all_idx, int(training_size / batch_size)):
        # Forward propagation
        Y_hat, z, a = forward_prop(train_images[batch_idx], w1,w2,w3,b1,b2,b3)
        # Back propagation
        g_w, g_b = back_prop(Y_hat,train_labels[batch_idx],[w1,w2,w3],[b1,b2,b3],a,z)
        #print(g_w, g_b)
        # Gradient update
        w1 -= alpha * g_w[0]
        w2 -= alpha * g_w[1]
        w3 -= alpha * g_w[2]
        b1 -= alpha * g_b[0]
        b2 -= alpha * g_b[1]
        b3 -= alpha * g_b[2]

    # After an epoch
    # Testing for accuracy
    w = [w1,w2,w3]
    b = [b1,b2,b3]
    loss_and_acc(train_images, train_labels, w, b)
    print("---test--")
    loss_and_acc(test_images, test_labels, w, b)


### 4. Plot
# for example
# plt.figure(figsize=(12,5))
# ax1 = plt.subplot(111)
# ax1.plot(......)
# plt.xlabel(......)
# plt.ylabel(......)
# plt.grid()
# plt.tight_layout()
# plt.savefig('figure.pdf', dbi=300)

0
15.93
---test--
15.6
1
29.48
---test--
27.1
2
39.51
---test--
35.9
3
43.85
---test--
40.2
4
44.2
---test--
42.5
5
40.57
---test--
39.9
6
31.35
---test--
31.5
7
26.93
---test--
26.9
8
66.67
---test--
62.0
9
79.58
---test--
75.5
10
84.23
---test--
81.0
11
88.79
---test--
87.1
12
88.79
---test--
87.1
13
87.54
---test--
84.0
14
88.17
---test--
86.0
15
88.16
---test--
85.9
16
91.97
---test--
88.3
17
92.94
---test--
90.0
18
91.57
---test--
87.2
19
93.74
---test--
90.9
20
93.34
---test--
89.7
21
93.57
---test--
89.8
22
93.92
---test--
90.7
23
94.86
---test--
91.6
24
93.7
---test--
90.4
25
95.16
---test--
91.4
26
95.28
---test--
92.0
27
96.22
---test--
93.7
28
96.12
---test--
93.2
29
95.98
---test--
91.7
30
95.59
---test--
92.4
31
95.55
---test--
92.3
32
95.91
---test--
93.2
33
96.84
---test--
93.0
34
82.22
---test--
78.4
35
96.52
---test--
93.3
36
95.52
---test--
90.7
37
96.59
---test--
92.8
38
97.2
---test--
93.6
39
96.94
---test--
93.5
40
94.79
---test--
90.9
41
97.03
---test--
93.0
42
96

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in exp
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in less
  This is separate from the ipykernel package so we can avoid doing imports until


10.01
---test--
8.5
76
10.01
---test--
8.5
77
10.01
---test--
8.5
78
10.01
---test--
8.5
79
10.01
---test--
8.5
80
10.01
---test--
8.5
81
10.01
---test--
8.5
82
10.01
---test--
8.5
83
10.01
---test--
8.5
84
10.01
---test--
8.5
85
10.01
---test--
8.5
86
10.01
---test--
8.5
87
10.01
---test--
8.5
88
10.01
---test--
8.5
89
10.01
---test--
8.5
90
10.01
---test--
8.5
91
10.01
---test--
8.5
92
10.01
---test--
8.5
93
10.01
---test--
8.5
94
10.01
---test--
8.5
95
10.01
---test--
8.5
96
10.01
---test--
8.5
97
10.01
---test--
8.5
98
10.01
---test--
8.5
99
10.01
---test--
8.5
